%pip install jupyter-dash
%pip install pandas

In [1]:
# NOTE: Cell must finish running before the next cell is added to queue
from jupyter_dash import JupyterDash
# JupyterDash.infer_jupyter_proxy_config()

In [10]:
# from jupyter_dash import JupyterDash
import dash
import pandas as pd
from dash import Dash, dcc, html, Input, Output, callback, State, no_update, clientside_callback, MATCH, ALL, Patch
import regex as re
from dash.exceptions import PreventUpdate
from base64 import b64encode
from pprint import pprint
import os
import numpy as np
import cv2
import dash_bootstrap_components as dbc
from nist_database import MockDB, MSSQLDatabase
from video_tools import local_file_v
import json
import random
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash_extensions.javascript import Namespace
from datetime import datetime, timedelta
from dash_extensions.javascript import assign

with open('./nist-ai.json','r') as f:
    AUTH = json.load(f)
DB=MockDB(AUTH,'NIST_AI') # Jupyter: MockDB | MSSQLDatabase

In [3]:
db = DB.execute_query("select * from video")
db

,id,filename,checksum,metadata
0,31,assets/data/TrackAddict/Log-20230626-173441 wa...,30b55d42d2d5a76d46a4387e4a5b55a2,"{""streams"": [{""index"": 0, ""codec_name"": ""h264""..."
1,33,assets/data/TrackAddict/Log-20230711-174746 Se...,3d7e8f32590e6056970c05626195c1a8,"{""streams"": [{""index"": 0, ""codec_name"": ""h264""..."
2,42,assets/data/TrackAddict/Log-20230707-093013 wa...,b9fbf97f0c79899a4f8af1304a245624,"{""streams"": [{""index"": 0, ""codec_name"": ""h264""..."


## Dash App

In [4]:
stylesheets = [dbc.themes.BOOTSTRAP] # carousel bootstrap
scripts = ["https://cdnjs.cloudflare.com/ajax/libs/chroma-js/2.1.0/chroma.min.js"]  # js lib used for colors
app = JupyterDash(__name__, external_scripts=scripts, external_stylesheets=stylesheets, prevent_initial_callbacks=True)

app.layout = html.Div([
    html.Div([

        # settings modal
        html.Div(children=[
            html.Div(children=[
                html.Button(children=[
                    html.Img(src='assets/icons/x-out.png'),
                ], id="close-modal", className='close-button'),
                html.H1('settings'),
                html.Label(children = [
                    'Query Result',
                    dcc.RadioItems(['video', 'thumbnails', 'map'], 'video', id='query-result-type'),
                ]),
            ], id='settings-content'),
        ], id='settings'),
        
        # aside
        html.Div(children=[

            # search bar
            html.Div([
                # dcc.Input with on input)
                dcc.Input(id='search-bar', value='', type='text'),
                html.Button('Search', id='submit'),
                html.Button(children=[
                    html.Img(src='./assets/icons/hamburger.png')
                ], id='settings-button', className='open-button')
            ], id='search-bar-submit'),

            # metadata
            html.Div([
                html.Button([
                    html.Img(src='./assets/icons/x-out.png', className='plus')
                ], id='plus-container'),
            ], id='metadata-tags'),

            # search results
            html.Div(id='err'),
            html.Div(id='search-results'),
        ], id='aside'),
        
        # content
        html.Div(children=[
            html.Video(src='', controls=True, id='video', autoPlay=True, muted=True, loop=True),
            html.Div([
                html.Div([], id='thumbnail-container', **{'data-source-idx': ''})
            ], className='buffer'),
            html.Div([], className='map-container'),
        ], id='content', **{'data-error': '', 'data-src': 'assets/test_data/pod_clip_1.mp4', 'data-time-start': '0'}),
    ], id='container'),

    # testing
    html.Div([
        html.Div(['string data'], id='one'),
        html.Div(['string data'], id='two'),
    ], id='test'),
])



## Metadata Tag Callbacks

In [5]:
# add & delete tags with '+' and 'x'
@callback(
    Output('metadata-tags', 'children'),
    Input('plus-container', 'n_clicks'),
    Input({'type': 'delete-container', 'index': ALL}, 'n_clicks'),
    State('metadata-tags', 'children'),
    prevent_initial_call=True,
)
def add_delete_tag(n_add_clicks, n_delete_clicks_list, tags):
    if dash.callback_context.triggered_id == 'plus-container':
        if not n_add_clicks: raise PreventUpdate
        patch_children = Patch()
        idx_to_append = len(tags) - 1

        patch_children.insert(idx_to_append, 
            html.Div([
                html.Label([
                    dcc.Input('', size='3', id={'type': 'metadata-key', 'index': n_add_clicks}, className='key', placeholder='key'),
                ], id={'type': 'metadata-key-label', 'index': n_add_clicks}, className="input-sizer", **{'data-value': ''}),
                html.Label([
                    dcc.Input('', size='3', id={'type': 'metadata-val', 'index': n_add_clicks}, className='val', placeholder='val'),
                ], id={'type': 'metadata-val-label', 'index': n_add_clicks}, className="input-sizer", **{'data-value': ''}),
                html.Button([
                    html.Img(src='./assets/icons/x-out.png', id=f'delete-{n_add_clicks}', className='delete')
                ], id={'type': 'delete-container', 'index': n_add_clicks}, className='delete-container rotate90')
            ], className='metadata-tag', id={'type': 'metadata-tag', 'index': n_add_clicks}),
        )
        return patch_children
    elif dash.callback_context.triggered_id['type'] == 'delete-container':
        el_id = dash.callback_context.triggered_id['index']
        patch_children = Patch()
        idx_to_del = -1
        for idx, tag in enumerate(tags):
            if tag['props']['id']['index'] == el_id:
                idx_to_del = idx
                break

        if idx_to_del == -1: raise PreventUpdate
        if not n_delete_clicks_list[idx_to_del]: raise PreventUpdate
        del patch_children[idx_to_del]
        return patch_children
    else:
        print('got here, idk my id is', dash.callback_context.triggered_id)
        return no_update

# make metadata tag inputs grow dynamically
clientside_callback(
    """function(value) { return value; }""",
    Output({"type": "metadata-key-label", "index": MATCH}, "data-value"),
    Input({"type": "metadata-key", "index": MATCH}, "value"),
)
clientside_callback(
    """function(value) { return value; }""",
    Output({"type": "metadata-val-label", "index": MATCH}, "data-value"),
    Input({"type": "metadata-val", "index": MATCH}, "value"),
)

## Search -> Query Results

In [6]:
def highlight(text, search):
    els = re.split(f'(\\b{search})', text, flags=re.IGNORECASE)
    for i in range(1, len(els), 2):
        els[i] = html.Span(els[i], className='highlight')
    return els

# convert seconds to hours:minutes:seconds
def format_time(time):
    hours = int(time // 3600)
    minutes = int((time - hours * 3600) / 60)
    seconds = int(time - hours * 3600 - minutes * 60)
    if hours: return f'{hours}:{minutes:02d}:{seconds:02d}'
    return f'{minutes}:{seconds:02d}'

# get query results, based on search & metadata
@callback(
    Output(component_id='search-results', component_property='children'),
    Output(component_id='err', component_property='children'),
    Input(component_id='submit', component_property='n_clicks'),
    State(component_id='metadata-tags', component_property='children'),
    State(component_id='search-bar', component_property='value'),
)
def update_output_div(n_clicks, tags, input_value):
    if not input_value: raise PreventUpdate
    if re.search(r'[.].+', input_value): return '', 'multiple sentences are not yet supported'

    # get metadata from #metadata-tags & add to query
    metadata = []
    for tag in tags[:-1]:
        key = tag['props']['children'][0]['props']['children'][0]['props']['value']
        val = tag['props']['children'][1]['props']['children'][0]['props']['value']
        # get all keys, not just first
        if key and val: metadata.append((key, val))
    metadata_query = 'select * from video where 1=1'
    for key, val in metadata:
        metadata_query += f" and JSON_VALUE(metadata, '$.{key}') like '%{val}%'"
    valid_vids = DB.execute_query(metadata_query)
    valid_vid_ids = valid_vids['id'].tolist()
    valid_vid_ids = [str(vid_id) for vid_id in valid_vid_ids]
    valid_video_id_query = f" and video_id in ({', '.join(valid_vid_ids)})" if valid_vid_ids else " and 1=0"
    
    # query
    df = DB.execute_query(f"select * from text_segment where segment like '%[^a-zA-Z]{input_value}%'" + valid_video_id_query)
    
    # return results as html
    ret = []
    for video_id, segment_id, word, time_start, thumbnail_bin in zip(df['video_id'], df['id'], df['segment'], df['time_start'], df['thumbnail']):
        thumbnail_src = 'https://listingsnearby.com/wp-content/uploads/2021/05/thumbnail-default-image.png' # Jupyter: f'data:image/png;base64,{b64encode(thumbnail_bin).decode()}' if thumbnail_bin else 'https://listingsnearby.com/wp-content/uploads/2021/05/thumbnail-default-image.png'
        video_src = DB.execute_query(f"select * from video where id={video_id}")['filename'].iloc[0]
        ret.append(
            html.Button(children=[
                html.Img(src=thumbnail_src, className='thumbnail-image'),
                html.Div(children=highlight(word, input_value), className='thumbnail-text'),
                html.Div(children=format_time(time_start), className='thumbnail-time')
            ], n_clicks=0, className='search-result', id={'type': 'search-result', 'index': segment_id}, **{'data-video-file': video_src, 'data-video-id': video_id, 'data-time-start': int(time_start)})
        )
    return ret, ''



## Query Result Click -> Content

In [7]:


# update video src on search result click
@callback(
    [Output('content', 'data-src'), Output('content', 'data-time-start'), Output('content', 'children')],
    [Input({'type': 'search-result', 'index': ALL}, 'n_clicks')],
    [State({'type': 'search-result', 'index': ALL}, 'id'),
    State({'type': 'search-result', 'index': ALL}, 'data-video-file'),
    State({'type': 'search-result', 'index': ALL}, 'data-video-id'),
    State({'type': 'search-result', 'index': ALL}, 'data-time-start'),
    State('query-result-type', 'value')],
    prevent_initial_call=True,
)
def update_video_src(n_clicks, ids, vid_files, vid_ids, time_starts, result_type):
    if not any([bool(n) for n in n_clicks]): raise PreventUpdate
    trigger_obj = dash.callback_context.triggered[0]['prop_id']
    trigger_id = json.loads(trigger_obj.split('.')[0])['index']

    video_id = -1
    for i in range(len(ids)):
        if ids[i]['index'] == trigger_id: 
            video_id = vid_ids[i]
            break
    
    # check vid_files[i] exists
    local_vid_src = local_file_v(vid_files[i])
    if not os.path.exists(local_vid_src): 
        print('could not fulfill request for video', vid_files[i], 'at time', time_starts[i])
        return ['', '', html.Div(f'Video not found at "{local_vid_src}"' , className='error')]

    # return video
    dash_video = html.Video(src=local_vid_src, controls=True, id='video', autoPlay=True, muted=False, loop=True)
    
    # return thumbnails
    deltas = np.linspace(-10000, 10000, 21) # +- 10 frames, +- 10 seconds
    abs_times = [time_starts[i] * 1000 + delta for delta in deltas]
    abs_times = [t for t in abs_times if t >= 0]
    source_idx = abs_times.index(time_starts[i] * 1000)
    vidcap = cv2.VideoCapture(local_vid_src)
    frames = []
    for time in abs_times:
        vidcap.set(cv2.CAP_PROP_POS_MSEC, time)
        success, image = vidcap.read()
        if not success: 
            print(f'could not read frame at {time} ms')
            continue
        success, buffer = cv2.imencode('.jpg', image)
        frames.append(f'data:image/png;base64,{b64encode(buffer).decode()}') # src's
    assert len(frames) == len(abs_times)
    dash_thumbnails = html.Div([
        html.Div([
            # button
            html.Button([
                html.Img(src=image, className='thumbnail-image')
            ], id={'type': 'thumbnail-button', 'index': i}, **{'data-video-seconds': abs_times[i] // 1000}, className='thumbnail-button' if i != source_idx else 'thumbnail-button source-idx')
            for i, image in enumerate(frames)
        ], id='thumbnail-container', **{'data-source-idx': source_idx})
    ], className='buffer')
    
    # return map
    # get bookend timestamps
    video_creation_time = DB.execute_query(f"select * from gps")[['timestamp']].iloc[0][0]
    text_segment_time = video_creation_time + timedelta(seconds=time_starts[i])
    begin_time = text_segment_time - timedelta(seconds=60)
    end_time = text_segment_time + timedelta(seconds=60)

    # get gps points
    gps_points = DB.execute_query(f"select * from gps where video_id={video_id} and timestamp between '{begin_time}' and '{end_time}'")
    latitudes, longitudes = gps_points['latitude'].tolist(), gps_points['longitude'].tolist()
    if len(latitudes) == 0: return [no_update, no_update, html.Div('No GPS data found for this video', className='error')]

    colorscale = ['red', 'yellow', 'green', 'blue', 'purple']  # rainbow
    color_prop = 'times'
    df = gps_points
    df = df[['timestamp', 'latitude', 'longitude', 'altitude']]
    df = df.rename(columns={'latitude': 'lat', 'longitude': 'lon'})
    times = list(df['timestamp'])
    times = [int((t - text_segment_time).total_seconds()) for t in times]
    df['times'] = times
    dicts = df.to_dict('rows')
    for item, time in zip(dicts, times):
        item["tooltip"] = f"{time:+} secs" # bind tooltip
    geojson = dlx.dicts_to_geojson(dicts, lon="lon")  # convert to geojson
    geobuf = dlx.geojson_to_geobuf(geojson)  # convert to geobuf
    # Create a colorbar.
    colorbar = dl.Colorbar(colorscale=colorscale, width=20, height=150, min=min(times), max=max(times), unit='sec')
    ns = ns = Namespace("myNamespace", "mySubNamespace")

    # Create geojson.
    geojson = dl.GeoJSON(data=geobuf, id="geojson", format="geobuf",
                        zoomToBounds=True,  # when true, zooms to bounds when data changes
                        options=dict(pointToLayer=ns("pointToLayer2")),  # how to draw points
                        superClusterOptions=dict(radius=50),   # adjust cluster size
                        hideout=dict(colorProp=color_prop, circleOptions=dict(fillOpacity=1, stroke=False, radius=5),
                                    min=min(times), max=max(times), colorscale=colorscale))

    n, m = len(latitudes), len(longitudes)
    mid_lat, mid_lon = latitudes[n // 2], longitudes[m // 2]
    print('found', len(latitudes), 'gps points')

    
    dash_map = html.Div([
        dl.Map([dl.TileLayer(), geojson, colorbar]),
    ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block", "position": "relative"}, className='map-container') # style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block", "position": "relative"}, className='map-container'

    return [vid_files[i], time_starts[i], html.Div([dash_video, dash_thumbnails, dash_map], className='buffer')]
    


## Content Callbacks

In [8]:
# every time new output is created in content (right-hand column), make sure new thumbnail container is responsive
clientside_callback(
    """ function(children, sourceIdx) {
        let thumbnails = document.getElementById('thumbnail-container');
        console.log('thumbnail container', thumbnails)
        if (thumbnails) {

            // get data
            let bc = thumbnails.getBoundingClientRect();
            let bcOffset = bc.left;
            let bcWidth = bc.width;
            let scrollWidth = thumbnails.scrollWidth;
            
            let containerWidth = thumbnails.offsetWidth;
            let numChildren = thumbnails.childElementCount;
            let childWidth = scrollWidth / numChildren; // takes into consideration gap between elements, albeit approximate.
            console.log('child width', childWidth);
            let pxOfElsOnLeft = sourceIdx * childWidth;

            console.log(bcWidth, containerWidth);

            // scroll based on data-source-idx attribute
            console.log('pxOfElsOnLeft', pxOfElsOnLeft, 'half container width', bcWidth / 2, 'half child width', childWidth / 2, 'all together', pxOfElsOnLeft - bcWidth / 2 + childWidth / 2)
            thumbnails.scrollLeft = Math.round(pxOfElsOnLeft - bcWidth / 2 + childWidth / 2);

            // make responsive
            thumbnails.addEventListener('mousemove', function(e) {
                let distIntoBox = e.pageX - bcOffset;
                let percentScroll = distIntoBox / bcWidth;
                thumbnails.scrollLeft = percentScroll * scrollWidth - bcWidth / 2;
            });
        }
        return [];
    }
    """,
    Output('one', 'children'),
    Input('content', 'children'),
    State('thumbnail-container', 'data-source-idx'),
)

# make thumbnail container click change the time of video player
clientside_callback(
    """ function(n_clicks) {
        // get the id of the button that triggered the callback
        let el = window.dash_clientside.callback_context.triggered
        if (el.length == 0) return [];
        let id_index_str = el[0].prop_id.split('.')[0];
        let thumbnail_buttons = document.getElementsByClassName('thumbnail-button');

        // extract the data attribute from the video id
        let video_seconds = -1;
        for (let i = 0; i < thumbnail_buttons.length; i++) {
            if (thumbnail_buttons[i].id == id_index_str) {
                video_seconds = thumbnail_buttons[i].getAttribute('data-video-seconds');
            }
        }

        // change the video based on data attribute
        vid = document.getElementById('video');
        if (vid) {
            console.log('video found:', vid);
            vid.currentTime = video_seconds;
            vid.pause();
            vid.muted = false; // still doesn't work
        } else {
            console.log('vid not found');
        }
        return [];
        
    }""",
    Output('two', 'children'),
    Input({'type': 'thumbnail-button', 'index': ALL}, 'n_clicks'),
)

# update time of video player when the data-time-start attribute changes
clientside_callback(
    """function(new_time, content_children) {
        console.log('updating video time')
        vid = document.getElementById('video');
        if (vid) {
            vid.currentTime = new_time;
            vid.play();
            vid.muted = false; // still doesn't work
        } else {
            console.log('vid not found');
        }
        return [];
    }""",
    Output('video', 'children'),
    Input('content', 'data-time-start'),
    State('content', 'children'),
    prevent_initial_call=False,
)

# toggle modal visibility
clientside_callback(
    """function(n_clicks_open, n_clicks_close, children) {
        console.log('hello from the callback 3');
        modal = document.getElementById('settings');
        if (!modal) {return children; }
        if (modal.style.display == 'flex') {
            modal.style.display = 'none';
        } else {
            modal.style.display = 'flex';
        }
        return children;
    }""",
    Output('settings', 'children'),
    Input('settings-button', 'n_clicks'),
    Input('close-modal', 'n_clicks'),
    State('settings', 'children'),
    prevent_initial_call=True,
)


In [9]:
app.run_server(mode='external', debug=True, port=8153) # Jupyter: app.run(mode='external') | app.run_server(debug=False, mode="jupyterlab")

Dash app running on http://127.0.0.1:8153/


found 27 gps points
